# Data Preprocessing

Min-max scaling of our features

In [26]:
import pandas as pd
import numpy as np  
import os

In [27]:
# For reproducibility of results
np.random.seed(0)

# Load the dataset if selected_features.csv does not exist
if os.path.exists("selected_features.csv"):
    selected_features = pd.read_csv("selected_features.csv")
    print(selected_features.head)
else:
    file_path = "rtfDataSet.csv"
    original_data = pd.read_csv(file_path)
    # Get only our features of interest and save them to a local file "selecte_features.csv"
    # 2 7 8 11 14 15 17 18 21 22 23 25 30 31 33 36 40 43 44 50 53 54 56 57 58 59 60 64 66 68
    selected_features = original_data.iloc[:, [2,7,8,11,14,15,17,18,21,22,23,25,30,31,33,36,40,43,44,50,53,54,56,57,58,59,60,64,66,68]]    
    # Save the selected features to a local file
    selected_features.to_csv("selected_features.csv", index=False)
    print(original_data.head)

<bound method NDFrame.head of             2         7         8        11        14        15        17  \
0   -0.684000 -0.113000  0.401000 -0.251000 -0.178000  0.321000  0.016000   
1   -0.529000 -0.066000 -0.168000 -0.205000  0.020000  0.031000 -0.165000   
2   -0.240000  0.031000 -0.077000 -0.036000 -0.005000  0.170000  0.212000   
3   -0.062000 -0.143000  0.068000 -0.295000 -0.188000 -0.317000 -0.113000   
4   -0.203000 -0.168000  0.008000  0.166000 -0.048000 -0.291000 -0.091000   
..        ...       ...       ...       ...       ...       ...       ...   
391 -0.240811  0.160049 -0.172790  0.431347  0.430861 -0.305451 -0.119770   
392 -0.179418 -0.000023 -0.260852  0.237564 -0.080075 -0.354038 -0.004051   
393 -0.553610 -0.005744 -0.068708 -0.253044  0.017528 -0.070628 -0.172178   
394  0.203276 -0.036268 -0.253699  0.382949  0.295444  0.049701 -0.245894   
395 -0.221108  0.450663 -0.293762  0.042793  0.153844 -0.268618 -0.083691   

           18        21        22  ...       

In [28]:
print(original_data)

            0         1         2         3         4         5         6  \
0   -0.536000  0.292000 -0.684000  0.123000 -0.118000  0.346000 -0.308000   
1   -0.496000 -0.298000 -0.529000  0.073000 -0.072000  0.107000 -0.316000   
2    0.008000 -0.031000 -0.240000  0.178000 -0.120000  0.317000 -0.481000   
3   -0.188000 -0.180000 -0.062000 -0.104000 -0.136000 -0.061000 -0.216000   
4    0.038000  0.155000 -0.203000 -0.088000 -0.084000 -0.164000 -0.145000   
..        ...       ...       ...       ...       ...       ...       ...   
391  0.074967 -0.127350 -0.240811 -0.037584  0.035714 -0.431190 -0.154309   
392  0.664006 -0.028111 -0.179418 -0.070884  0.048793 -0.001349 -0.187850   
393 -0.234197 -0.058229 -0.553610 -0.097793 -0.157780  0.226956  0.416572   
394  0.274072 -0.105900  0.203276 -0.061067 -0.100202 -0.404380 -0.085191   
395 -0.357250 -0.068535 -0.221108 -0.036466 -0.086490 -0.069679  0.557720   

            7         8         9  ...        61        62        63  \
0  

In [29]:
labels = original_data.iloc[:,70]
print(labels)

0      0
1      0
2      0
3      0
4      1
      ..
391    1
392    1
393    1
394    1
395    1
Name: label, Length: 396, dtype: int64


## Min-Max

In [30]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
dataset_normalized = scaler.fit_transform(selected_features)
print(dataset_normalized)

[[0.18304732 0.49900892 0.89076305 ... 0.33932274 0.34881517 0.52682455]
 [0.26365055 0.52229931 0.43373494 ... 0.38908086 0.44834123 0.54374094]
 [0.41393656 0.5703667  0.50682731 ... 0.37456807 0.50900474 0.43547608]
 ...
 [0.25085283 0.55215857 0.51348755 ... 0.31353283 0.41787867 0.28746834]
 [0.6444493  0.53703271 0.3649004  ... 0.57430339 0.55802938 0.67151474]
 [0.42376079 0.77832656 0.33272129 ... 0.38708569 0.42213365 0.63412131]]


## Stratified 10-Fold Cross-Validation

In [31]:
from sklearn.model_selection import StratifiedKFold
# from sklearn.naive_bayes import GaussianNB


# TODO: Initialize your features as X and labels as y
X = dataset_normalized
y = labels

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=10)
# nb = GaussianNB()


# TODO: Implement the loop for Stratified 10-Fold CV
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # TODO: Insert your classifier training and testing code here
    # nb.fit(X_train, y_train)
    # print("\n\nPredicted: ")
    # print(nb.predict(X_test))
    # print("\nReal: ")
    # print(y_test)
    

## Naive Bayes

In [32]:
from sklearn.naive_bayes import GaussianNB

# Initialize classifier
nb = GaussianNB()

# TODO: Fit the model with training data
# nb.fit(X_train, y_train)

# TODO: Make predictions on test data
# y_pred = nb.predict(X_test)


## k-Nearest Neighbors Classifier

In [33]:
from sklearn.neighbors import KNeighborsClassifier

# TODO: Initialize classifier with k value and metric type
# knn = KNeighborsClassifier(n_neighbors=k, metric='euclidean')


# TODO: Fit the model with training data
# knn.fit(X_train, y_train)

# TODO: Make predictions on test data
# y_pred = knn.predict(X_test)


In [35]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Define the knn_classifier function
def knn_classifier(X_train, X_test, y_train, y_test, k_values, distance_metrics):
    results = {}

    for distance_metric in distance_metrics:
        best_k = None
        best_accuracy = 0

        for k in k_values:
            # Create an instance of the k-NN classifier with the specified distance metric and k value.
            knn_classifier = KNeighborsClassifier(n_neighbors=k, metric=distance_metric)

            # Train the classifier with the training data.
            knn_classifier.fit(X_train, y_train)

            # Make predictions on the test data.
            y_pred = knn_classifier.predict(X_test)

            # Calculate the classifier's accuracy.
            accuracy = accuracy_score(y_test, y_pred)

            # If we find a value of k with better accuracy, update the optimal values.
            if accuracy > best_accuracy:
                best_k = k
                best_accuracy = accuracy

        results[distance_metric] = {'best_k': best_k, 'best_accuracy': best_accuracy}

    return results

In [36]:
# Call the knn_classifier function
results = knn_classifier(X_train, X_test, y_train, y_test, k_values, distance_metrics)

# Print or analyze the results
for distance_metric, result in results.items():
    best_k = result['best_k']
    best_accuracy = result['best_accuracy']
    print(f"Best k for {distance_metric} distance metric: {best_k}")
    print(f"Accuracy for best k: {best_accuracy}\n")

Best k for euclidean distance metric: 5
Accuracy for best k: 0.8717948717948718

Best k for manhattan distance metric: 3
Accuracy for best k: 0.8461538461538461

